Note that any backslashes in this notebook may need to be changed to a forward slash depending on the operating system. This cell sets the name of directories where the data is stored.

In [72]:
index = collect(findlast("\\", pwd()))[end]
parent_dir = pwd()[1:index]
cross_section_dir = "C:\\Cross-Section-Data\\"
import_data_dir = parent_dir * "ImportedData\\"
export_data_dir = parent_dir * "ExportedData\\"
endf8_decay_dir = cross_section_dir * "ENDF-B-VIII.0\\decay\\"
jeff33_decay_dir = cross_section_dir * "ENDF_Libraries-2\\JEFF-3.3\\decay\\"
jendl5_decay_dir = cross_section_dir * "jendl5-dec_upd5\\"

"C:\\Cross-Section-Data\\jendl5-dec_upd5\\"

Import packages

In [73]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy #https://docs.juliahub.com/SymPy/ 
using PDFIO
using Unzip
using Interpolations
using Plots
using Downloads
#plotlyjs()
P, N, x, Σ, σR, σT, ρ, M, A = symbols("P, N, x, Σ, σ_R, σ_T, ρ, M, A", 
                                        positive = true, real = true)
N = ρ * A/ M
Σ = N * σT  
P = integrate(Σ*exp(-Σ*x), (x, 0, x)) * σR / σT

    ⎛     -A⋅x⋅ρ⋅σ_T ⎞
    ⎜     ───────────⎟
    ⎜          M     ⎟
σ_R⋅⎝1 - ℯ           ⎠
──────────────────────
         σ_T          

Now I will find the average decay energies for each nuclide. For the datasets I am using, the average decay energies are all given in keV. 

In [115]:
function get_mass_name(nuclide)
    index = 1
    while (tryparse(Int64, string(nuclide[index])) != nothing && index <= length(nuclide))
        index += 1
    end
    mass = tryparse(Int64, nuclide[1:index-1])
    name =  nuclide[index:end]
    name_1 = uppercase(name[1])
    if (length(name) > 1)
        name = name_1 * name[2]
    else 
      name = name_1
    end
    return mass, name
end
element_symbols = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", 
"Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", 
"Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", 
"Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", 
"Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", 
"Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", 
"Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", 
"Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", 
"Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", 
"Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"];
function atomic_number(symbol)
    index = 1
    while symbol != element_symbols[index]
        index += 1
        if index > length(element_symbols)
            return "Element not in periodic table"
        end
    end
    return index 
end

atomic_number (generic function with 1 method)

This dictionary stores what strings to look to for in the datasets to find the average energy for each decay type, and for each directory. 

In [75]:
dir_search_string_dict = Dict([
    jeff33_decay_dir => Dict(["alpha" => "ALPHA ENERGY                   =",
    "beta-minus" => "MEAN BETA- ENERGY              =","gamma" => "MEAN GAMMA ENERGY              =", 
    "x-ray" => "MEAN X-RAY ENERGY              =", "auger" => "MEAN AUGER ELECTRON ENERGY     ="
    ]),

    endf8_decay_dir => Dict(["alpha" => "Mean Alpha Energy:", 
    "beta-minus" => "Mean B- Energy:", "beta-plus" => "Mean B+ Energy:",
    "gamma" => "Mean Gamma Energy:", "auger" => "Mean CE+Auger Energy:",
    "neutron" => "Mean Neutron Energy:", "proton" => "Mean Neutron Energy:",
    "half life" => "Parent half-life:"]),

    jendl5_decay_dir => Dict(["alpha" => "Mean Alpha Energy",
    "beta-minus" => "Mean B- Energy", "gamma" => "Mean Gamma Energy",
    "neutron" => "Mean Neutron Energy", "proton" => "Mean Proton Energy",
    "auger" => "Mean CE+Auger Energy", 
    "beta-plus and electron capture" => "Mean B+/EC Energy"])  
])

Dict{String, Dict{String, String}} with 3 entries:
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy", "beta-plus…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"MEAN AUGER ELECTRON ENERGY     ="…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy:", "proton"=…

In [76]:
function get_decay_energy_from_file(file_as_arr, search_string)
    line = [l for l in file_as_arr if length(l) > length(search_string)
             && occursin(search_string, l)]
    if (length(line) > 0)
        line = String(line[1])
        line = join(split(line, search_string, keepempty = false))
        return tryparse(Float64, join(split(line, " ", keepempty = false)[1]))
    else 
        return "Energy not found"
    end
end

get_decay_energy_from_file (generic function with 1 method)

This function is meant for particularly short half lives reported in terms of the decay width. 

small_decay_time(nuclide, dir) = string(half_life_sec(find_decay_info(nuclide, 
                                        "half life", dir)))   
small_decay_time("4H", endf8_decay_dir)

In [78]:
function find_decay_info(nuclide, info, dir)
    a, el = get_mass_name(nuclide)
    el = string(el)
    z = [i for i in 1:length(element_symbols) if element_symbols[i] == el][1]
    file_name = string(z) * "-" * el * "-" * string(a) * ".dat"
    if !(file_name in readdir(dir))
        return "insufficient data"
    end
    file_path = dir * file_name
    file_as_array = split(open(f->read(f, String), file_path), "\n")
    search_string = dir_search_string_dict[dir][info]
    return get_decay_energy_from_file(file_as_array, search_string)
end

find_decay_info (generic function with 1 method)

In [79]:
function find_avg_decay_energy(nuclide, decay_type, dir)
    a, el = get_mass_name(nuclide)
    el = string(el)
    z = [i for i in 1:length(element_symbols) if element_symbols[i] == el][1]
    file_name = string(z) * "-" * el * "-" * string(a) * ".dat"
    if ! (file_name in readdir(dir))
        return "Energy not found"
    end
    file_path = dir * file_name
    file_as_array = split(open(f->read(f, String), file_path), "\n")
    search_string = dir_search_string_dict[dir][decay_type]
    return get_decay_energy_from_file(file_as_array, search_string)
end
dir_list = keys(dir_search_string_dict)
function find_avg_decay_energy(nuclide, decay_type)
    if decay_type == "electron capture"
        decay_type = "beta-plus and electron capture"
    end
    for dir in dir_list
        if decay_type in keys(dir_search_string_dict[dir])
            energy = find_avg_decay_energy(nuclide, decay_type, dir)
            if energy != "Energy not found"
                return energy
            end
        end
    end
    return "Energy not found"
end

find_avg_decay_energy (generic function with 2 methods)

In [80]:
joules_per_keV = Rational(uconvert(@u_str("J"), 1 * @u_str("keV")) / @u_str("J"))
joules_per_MeV = 1000 * joules_per_keV
h = 6.62607015 * 10.0^(-34) / (2 * pi)
half_life_sec(gamma) = h / (joules_per_MeV * gamma)
half_life_sec(4.6)

1.4308955585889278e-22

Very short half lives are often reported in terms of gamma. Source: 

https://www.nndc.bnl.gov/nudat3/help/glossary.jsp#halflife

In [81]:
decay_or_emission_type_dict = Dict([
    "B-" =>  "beta minus decay", "A" =>  "alpha decay", "B+" =>  "beta plus decay",
    "EC" => "auger and conversion electron"
])
function get_half_life(nuclide, decay_type)
    if decay_type == "EC+B+" 
        decay_type = "B+"
    end
    if decay_type in keys(decay_or_emission_type_dict)
        decay_type_long = decay_or_emission_type_dict[decay_type]
        row = [row for row in 1:size(all_live_chart_data)[1]
                if ((all_live_chart_data[row, "nuclide"] ==  nuclide) && 
                (decay_type_long == all_live_chart_data[row, "Decay or Emission type"])
                )][1]
        return all_live_chart_data[row, "half_life_sec"]
    end
    if decay_type in values(decay_or_emission_type_dict)
        row = [row for row in 1:size(all_live_chart_data)[1]
                if (all_live_chart_data[row, "nuclide"] ==  nuclide && 
                decay_type == all_live_chart_data[row, "Decay or Emission type"])][1]
        return all_live_chart_data[row, "half_life_sec"]
    end
    if decay_type in keys(livechart_abbreviation_decay_types)
        row = 1
        while (row < size(all_live_chart_data)[1] && 
                all_live_chart_data[row, "nuclide"] !=  nuclide)
            row += 1
        end
        half_life =  all_live_chart_data[row, "half_life_sec"]
        if split(half_life, "second", keepempty = false)[1] == " "
            try 
                return small_decay_time(nuclide, endf8_decay_dir)
            catch
                return "insufficient data"
            end
        end
        return half_life
    end
    return decay_type * " not accounted for "
end

get_half_life (generic function with 1 method)

In [134]:
collect(findlast(".", "doggo.endf"))[1]

6

In [146]:
function get_metastable_file_paths(dir, file_type)
    l = length(file_type)
    decay_files = [x for x in readdir(dir) 
                    if length(x) > l && x[end-(l-1):end] == file_type]
    metastable_decay_files = [x for x in decay_files if occursin("m", x)]
    return [dir * file for file in metastable_decay_files]
end
function get_metastable_file_paths(dir)
    example_path = readdir(dir)[end-1]
    file_type_index = collect(findlast(".", example_path))[1]
    file_type = example_path[file_type_index:end]
    return get_metastable_file_paths(dir, file_type)
end
endf8_metstable_paths = get_metastable_file_paths(endf8_decay_dir)

906-element Vector{String}:
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-011_Na_024m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-013_Al_024m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-013_Al_026m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-017_Cl_034m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-017_Cl_038m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-019_K_038m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_042m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_044m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_045m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_046m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-021_Sc_050m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-023_V_044m1.endf"
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-023_V_046m1.endf"
 ⋮
 "C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\d

In [147]:
jendl5_metastable_paths = get_metastable_file_paths(jendl5_decay_dir)

971-element Vector{String}:
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-242.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-243.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-244.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-245.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-246.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-247.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-247m1.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-248.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-249.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-250.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-250m1.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-251.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\100-Fm-252.dat"
 ⋮
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\96-Cm-245.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\96-Cm-246.dat"
 "C:\\Cross-Section-Data\\jendl5-dec_upd5\\96-Cm-247.dat"
 "C:\\Cross-Section-Data

This function returns the name of an isotope given its file path in the ENDF8 directory. The last five characters in this directory are ".endf" and the first 8 characters are: "dec-", atomic number, and "_". Note that this is note the case in the ENDF-Libraries-2 (that version of ENDF8 decay data is not being used here because it is missing some metastabale isotopes).

Additionally, the file names in JENDL5 are not zero-padded, so this function accounts for that.

In [177]:
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O',
            'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
function isotope_name(file_path) 
    file = split(file_path, "\\")[end]
    seperation_characters = ['-', '_']
    relevant_indices = [i for i in 1:length(file) 
                        if !(file[i] in seperation_characters)]
    file = file[relevant_indices]
    period_index = 1
    while file[period_index] != '.' && period_index < length(file)
        period_index += 1
    end
    symbol_index = 2
    while !(file[symbol_index] in alphabet) && symbol_index < period_index
        symbol_index += 1
    end
    return split(file[symbol_index:period_index-1], "dec")[end]
end
isotope_name("C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-110_Ds_279m1.endf")

"Ds279m1"

In [178]:
isotope_name("C:\\Cross-Section-Data\\jendl5-dec_upd5\\97-Bk-248m1.dat")

"Bk248m1"

In [179]:
join(["dog", "cat"])

"dogcat"

Here m_isotope denotes a given metastable isotope. 
I will make it return the file names of all the ground states of the metastable isotopes in a given directory.

In [188]:
function ground_state_half_life(m_isotope_file_path)
    m_isotope = isotope_name(m_isotope_file_path)
    start_of_file_name = length(m_isotope_file_path)-1
    while m_isotope_file_path[start_of_file_name] != '\\'
        start_of_file_name -= 1
        if start_of_file_name < 1
            return "Must use full path of file; e.g. dir\\file.filetype"
        end
    end
    dir = m_isotope_file_path[1:start_of_file_name]
    file_type_index = collect(findlast(".", m_isotope_file_path))[1]
    file_type = example_path[file_type_index:end]
    ground_state = m_isotope[1:end-2]

        padded_atomic_number = lpad(string(atomic_number(ground_state[1:2])), 3, "0")
        #left pad the atomic number of the given isotope with zeroes
        ground_state_file_path = (endf8_decay_dir * "dec-" * padded_atomic_number *
                                "_" * m_isotope[1:2] * "_" * m_isotope[3:5] * ".endf")
    if !(ground_state_file_path in readdir(dir))
        #need to look in different directory
        return "No data for the ground state in the same directory."
    end

end
ground_state_half_life("C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\dec-110_Ds_279m1.endf")


C:\Cross-Section-Data\ENDF-B-VIII.0\decay\dec-110_Ds_279m1.endf

"C:\\Cross-Section-Data\\ENDF-B-VIII.0\\decay\\"